In [2]:
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup

In [3]:
base_url = "http://books.toscrape.com/index.html"
home_page = requests.get(base_url)
home_page.status_code

200

In [4]:
soup = BeautifulSoup(markup = home_page.content , parser = "html.parser")
books = soup.find_all(name = "li" , class_ = "col-xs-6 col-sm-4 col-md-3 col-lg-3")
len(books)

20

In [5]:
book = books[0]

In [6]:
book_url = book.findChild(name = "a").get("href")
book_url

'catalogue/a-light-in-the-attic_1000/index.html'

In [7]:
book_url = urljoin(base_url, book_url)
book_url

'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'

In [8]:
book_info = requests.get(book_url).content
book_soup = BeautifulSoup(markup = book_info , parser = "http.parser")

In [9]:
name = book_soup.find(name ="h1").getText()
name

'A Light in the Attic'

In [10]:
book_table_data = book_soup.find_all(name="tr")
book_table_data

[<tr>
 <th>UPC</th><td>a897fe39b1053632</td>
 </tr>,
 <tr>
 <th>Product Type</th><td>Books</td>
 </tr>,
 <tr>
 <th>Price (excl. tax)</th><td>£51.77</td>
 </tr>,
 <tr>
 <th>Price (incl. tax)</th><td>£51.77</td>
 </tr>,
 <tr>
 <th>Tax</th><td>£0.00</td>
 </tr>,
 <tr>
 <th>Availability</th>
 <td>In stock (22 available)</td>
 </tr>,
 <tr>
 <th>Number of reviews</th>
 <td>0</td>
 </tr>]

In [11]:
book_table_data.find(name="tr").getText()

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [12]:
book_table_data[0:2]

[<tr>
 <th>UPC</th><td>a897fe39b1053632</td>
 </tr>,
 <tr>
 <th>Product Type</th><td>Books</td>
 </tr>]

In [13]:
book_data = {}
for i in book_table_data:
  key = i.find(name="th").getText()
  value = i.find(name ="td").getText()
  book_data[key]= value


book_data

{'UPC': 'a897fe39b1053632',
 'Product Type': 'Books',
 'Price (excl. tax)': '£51.77',
 'Price (incl. tax)': '£51.77',
 'Tax': '£0.00',
 'Availability': 'In stock (22 available)',
 'Number of reviews': '0'}

In [33]:
def scrape_book(book_url):
  book_content = requests.get(book_url).content
  book_content_soup = BeautifulSoup(markup=book_content, parser="html.parser")

  #getting name of the book
  book_data ={}
  name = book_content_soup.find(name="h1").getText()
  book_data['name'] = name

  #getting other data
  book_table_data = book_content_soup.find_all(name="tr")
  for i in book_table_data:
    key = i.find(name="th").getText()
    value = i.find(name="td").getText()
    book_data [key] = value

  #adding the url at the end
  book_data['url'] = book_url

  return book_data


In [30]:
url ="http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"
scrape_book(url)

{'name': 'A Light in the Attic',
 'UPC': 'a897fe39b1053632',
 'Product Type': 'Books',
 'Price (excl. tax)': '£51.77',
 'Price (incl. tax)': '£51.77',
 'Tax': '£0.00',
 'Availability': 'In stock (22 available)',
 'Number of reviews': '0',
 'url': 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'}

In [43]:
def scrape_page(page_url):
  books_data = []
  page_content = requests.get(page_url).content
  page_soup = BeautifulSoup(markup = page_content , parser = "html.parser")
  page_books = page_soup.find_all(name="li", class_ = "col-xs-6 col-sm-4 col-md-3 col-lg-3")

  for i in page_books:
    book_url = i.findChild(name ="a").get("href")
    book_url = urljoin(page_url, book_url)
    book_data = scrape_book(book_url)
    books_data.append(book_data)

  return book_data

In [38]:
page_url = "https://books.toscrape.com/catalogue/page-1.html"
books_data = []
page_content = requests.get(page_url).content
page_soup = BeautifulSoup(markup = page_content , parser = "html.parser")
page_books = page_soup.find_all(name="li", class_ = "col-xs-6 col-sm-4 col-md-3 col-lg-3")
page_books[0]

<li class="col-xs-6 col-sm-4 col-md-3 col-lg-3">
<article class="product_pod">
<div class="image_container">
<a href="a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>
</li>

In [40]:
url = "https://books.toscrape.com/catalogue/page-1.html"
scrape_page(url)

MissingSchema: Invalid URL 'a-light-in-the-attic_1000/index.html': No scheme supplied. Perhaps you meant https://a-light-in-the-attic_1000/index.html?

In [60]:
page_count = 1
data = []

while True:
  page_url = f"https://books.toscrape.com/catalogue/page-{page_count}.html"
  status = requests.get(page_url).status_code

  #break the loop if exceed the total page count
  if status == 404 or page_count == 6:
    break

  page_data = scrape_page(page_url)
  data.append(page_data)
  print(f"page :{page_count} is SUCCESSFULLY scraped")

  page_count += 1

page :1 is SUCCESSFULLY scraped
page :2 is SUCCESSFULLY scraped
page :3 is SUCCESSFULLY scraped
page :4 is SUCCESSFULLY scraped
page :5 is SUCCESSFULLY scraped


In [61]:
data

[{'name': "It's Only the Himalayas",
  'UPC': 'a22124811bfa8350',
  'Product Type': 'Books',
  'Price (excl. tax)': '£45.17',
  'Price (incl. tax)': '£45.17',
  'Tax': '£0.00',
  'Availability': 'In stock (19 available)',
  'Number of reviews': '0',
  'url': 'https://books.toscrape.com/catalogue/its-only-the-himalayas_981/index.html'},
 {'name': "You can't bury them all: Poems",
  'UPC': '55f9da0c5eea2e10',
  'Product Type': 'Books',
  'Price (excl. tax)': '£33.63',
  'Price (incl. tax)': '£33.63',
  'Tax': '£0.00',
  'Availability': 'In stock (17 available)',
  'Number of reviews': '0',
  'url': 'https://books.toscrape.com/catalogue/you-cant-bury-them-all-poems_961/index.html'},
 {'name': 'The Natural History of Us (The Fine Art of Pretending #2)',
  'UPC': 'cedf82b5086e4691',
  'Product Type': 'Books',
  'Price (excl. tax)': '£45.22',
  'Price (incl. tax)': '£45.22',
  'Tax': '£0.00',
  'Availability': 'In stock (16 available)',
  'Number of reviews': '0',
  'url': 'https://books.tos

In [64]:
book = data[0].copy()

In [70]:
float(book['Price (excl. tax)'][1:])

45.17

In [75]:
book['Availability'].split("(")[-1]

'19 available)'

In [77]:
book['Availability'].split("(")[-1][:-1]

'19 available'

In [78]:
book['Availability'].split("(")[-1][:-1].split()[0]

'19'

In [79]:
quantity_available = int(book['Availability'].split("(")[-1][:-1].split()[0])

In [89]:
is_available = book['Availability'].split("(")[0].strip()

In [90]:
quantity_available , is_available

(19, 'In stock')

In [93]:
def fix(item):
  item['Price (excl. tax)'] = float(item['Price (excl. tax)'][1:])
  item['Price (incl. tax)'] = float(item['Price (incl. tax)'][1:])
  item['Tax'] = float(item['Tax'][1:])
  availablity = item.pop('Availability')
  item['is_available'] = True if availablity.split("(")[0].strip() == 'In stock' else False
  item['quantity_available'] = int(book['Availability'].split("(")[-1][:-1].split()[0])
  return item

In [94]:
formatted_data = [fix(item.copy()) for item in data]
formatted_data

[{'name': "It's Only the Himalayas",
  'UPC': 'a22124811bfa8350',
  'Product Type': 'Books',
  'Price (excl. tax)': 45.17,
  'Price (incl. tax)': 45.17,
  'Tax': 0.0,
  'Number of reviews': '0',
  'url': 'https://books.toscrape.com/catalogue/its-only-the-himalayas_981/index.html',
  'is_available': True,
  'quantity_available': 19},
 {'name': "You can't bury them all: Poems",
  'UPC': '55f9da0c5eea2e10',
  'Product Type': 'Books',
  'Price (excl. tax)': 33.63,
  'Price (incl. tax)': 33.63,
  'Tax': 0.0,
  'Number of reviews': '0',
  'url': 'https://books.toscrape.com/catalogue/you-cant-bury-them-all-poems_961/index.html',
  'is_available': True,
  'quantity_available': 19},
 {'name': 'The Natural History of Us (The Fine Art of Pretending #2)',
  'UPC': 'cedf82b5086e4691',
  'Product Type': 'Books',
  'Price (excl. tax)': 45.22,
  'Price (incl. tax)': 45.22,
  'Tax': 0.0,
  'Number of reviews': '0',
  'url': 'https://books.toscrape.com/catalogue/the-natural-history-of-us-the-fine-art-of